Separately run referrals and claims queries.
This file imports the results of those queries, cleans and standardizes each, and merges them for analysis.

In [194]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import pyodbc
import os

## Import Data

In [195]:
with open('referrals.sql', 'r') as myfile:
    refs_sql_str=myfile.read().replace('\n', ' ')

cnxn_refs = pyodbc.connect('DRIVER={SQL Server};SERVER=colo-dwrpt01;DATABASE=IADS_V3')

refs = pd.read_sql(refs_sql_str, cnxn_refs)

cnxn_refs.close()

In [196]:
with open('claims.sql', 'r') as myfile:
    claims_sql_str=myfile.read().replace('\n', ' ')

cnxn_claims = pyodbc.connect('DRIVER={SQL Server};SERVER=colo-dwrpt01;DATABASE=NATIONAL_ANALYTICS')

claims = pd.read_sql(claims_sql_str, cnxn_claims)

cnxn_claims.close()

In [197]:
with open('referrals_new.sql', 'r') as myfile:
    refs_new_sql_str=myfile.read().replace('\n', ' ')

cnxn_refs_new = pyodbc.connect('DRIVER={SQL Server};SERVER=colo-dwrpt01;DATABASE=IADS_V3')

refs_new = pd.read_sql(refs_new_sql_str, cnxn_refs_new)

cnxn_refs_new.close()

In [198]:
refs_new.shape[0]

952977

In [199]:
codes_to_pend = pd.read_csv('../data/V1_codes_to_approve.csv')

## Clean Refs

In [200]:
refs_head = refs.drop_duplicates(subset='HCP_CONNECT_AUTH_NUMBER').reset_index()

refs_head.drop(columns = ['index', 'CPT_Code', 'Date_Decision',
       'Date_Received', 'Type', 'region', 'UNITS'], inplace=True)

refs_head = refs_head[refs_head['HCP_CONNECT_AUTH_NUMBER'].isna()==False].reset_index()

In [201]:
refs['Date_Decision'] = pd.to_datetime(refs['Date_Decision'])

In [202]:
refs['Date_Received'] = pd.to_datetime(refs['Date_Received'])

### Clean Claims

In [203]:
## Claims sum is used for the cost of a cpt code when it doesn't exist in claims for the assocaited specialty
claims_sum = claims.groupby(['CPT_Code'], as_index=False).agg({'avg_hcp_cost': 'mean'})

## Feature Engineering

In [204]:
## flag retro statuses with 1 and 0
retro_conditions = [
 (refs['status_name'] == 'APPROVED - RETRO REVIEW') |
 (refs['status_name'] == 'DENIED - RETRO REVIEW') |
 (refs['status_name'] == 'APPROVED - COB RETRO') |
 (refs['status_name'] == 'PENDING - RETRO REVIEW') 
  ]

In [205]:
## Create Refs - the older set of referrals from which we make the rules

In [206]:
choices = [1]
refs['is_retro'] = np.select(retro_conditions, choices, default=0)

In [207]:
## remove retros from list and drop 'is_retro' as it is no longer needed
refs = refs[refs['is_retro']==0]

refs.drop(columns='is_retro', inplace=True)

In [208]:
refs_det = refs

refs_det.drop(columns = ['PPL', 'Date_Decision',
       'Date_Received', 'Type', 'status_cat', 'status_name', 'Specialty',
       'region', 'UNITS'])

refs_det = refs_det[refs_det['HCP_CONNECT_AUTH_NUMBER'].isna()==False].reset_index()

In [209]:
refs['is_autoapp'] = np.where(refs['status_name']=='APPROVED - AUTO', 1, 0)

In [210]:
refs.PPL.fillna("N", inplace=True)

In [211]:
refs['is_PPL'] = np.where(refs['PPL']=='Y', 1, 0)

In [212]:
den_conditions = [
 (refs['status_name'] == 'DENIED - CM') |
 (refs['status_name'] == 'DENIED - BENEFIT CARVE OUT') |
 (refs['status_name'] == 'DENIED - NOT A COVERED BENEFIT') |
 (refs['status_name'] == 'DENIED - APPEAL') |
 (refs['status_name'] == 'DENIED - CLINICAL TRIAL/EXP/INV') |
 (refs['status_name'] == 'DENIED - TRANSPLANT') |
 (refs['status_name'] == 'DENIED - MD') |
 (refs['status_name'] == 'DENIED - CM/MD') |
 (refs['status_name'] == 'DENIED - REDIRECT OSVN') |
 (refs['status_name'] == 'DENIED - TICKLER')
  ]

In [213]:
choices = [1]
refs['is_den'] = np.select(den_conditions, choices, default=0)

In [214]:
refs['is_app'] = np.where(refs['status_cat']=='APPROVED', 1, 0)

In [215]:
refs.columns

Index(['CPT_Code', 'HCP_CONNECT_AUTH_NUMBER', 'REF_TYPE_KEY', 'PPL',
       'Date_Received', 'Date_Decision', 'Type', 'status_cat', 'status_name',
       'Specialty', 'region', 'UNITS', 'is_autoapp', 'is_PPL', 'is_den',
       'is_app'],
      dtype='object')

In [216]:
refs.pivot_table(values='HCP_CONNECT_AUTH_NUMBER', index=['is_autoapp', 'is_den'], aggfunc='count', margins=True)

HCP_CONNECT_AUTH_NUMBER
is_autoapp is_den                         
0          0                       2485581
           1                         59086
1          0                        949624
All                                3494291

In [217]:
59086/3494258

0.0169094554552068

## Clean refs_new - a new set of referrals to apply the rules to.

In [218]:
refs_new['Date_Decision'] = pd.to_datetime(refs_new['Date_Decision'])

In [219]:
refs_new['Date_Received'] = pd.to_datetime(refs_new['Date_Received'])

In [220]:
## flag retro statuses with 1 and 0
retro_conditions_new = [
 (refs_new['status_name'] == 'APPROVED - RETRO REVIEW') |
 (refs_new['status_name'] == 'DENIED - RETRO REVIEW') |
 (refs_new['status_name'] == 'APPROVED - COB RETRO') |
 (refs_new['status_name'] == 'PENDING - RETRO REVIEW') 
  ]

In [221]:
choices = [1]
refs_new['is_retro'] = np.select(retro_conditions_new, choices, default=0)

In [222]:
## remove retros from list and drop 'is_retro' as it is no longer needed
refs_new = refs_new[refs_new['is_retro']==0]

refs_new.drop(columns='is_retro', inplace=True)

In [223]:
refs_new['is_autoapp'] = np.where(refs_new['status_name']=='APPROVED - AUTO', 1, 0)

In [224]:
refs_new.PPL.fillna("N", inplace=True)

In [225]:
refs_new['is_PPL'] = np.where(refs_new['PPL']=='Y', 1, 0)

In [226]:
refs_new['is_app'] = np.where(refs_new['status_cat']=='APPROVED', 1, 0)

In [227]:
refs_new['UNITS'] = 1

In [228]:
den_conditions_new = [
 (refs_new['status_name'] == 'DENIED - CM') |
 (refs_new['status_name'] == 'DENIED - BENEFIT CARVE OUT') |
 (refs_new['status_name'] == 'DENIED - NOT A COVERED BENEFIT') |
 (refs_new['status_name'] == 'DENIED - APPEAL') |
 (refs_new['status_name'] == 'DENIED - CLINICAL TRIAL/EXP/INV') |
 (refs_new['status_name'] == 'DENIED - TRANSPLANT') |
 (refs_new['status_name'] == 'DENIED - MD') |
 (refs_new['status_name'] == 'DENIED - CM/MD') |
 (refs_new['status_name'] == 'DENIED - REDIRECT OSVN') |
 (refs_new['status_name'] == 'DENIED - TICKLER')
  ]

In [229]:
choices = [1]
refs_new['is_den'] = np.select(den_conditions_new, choices, default=0)

In [230]:
refs_new = refs_new[refs_new['HCP_CONNECT_AUTH_NUMBER'].isna()==False].reset_index()

In [231]:
refs_new.pivot_table(values='HCP_CONNECT_AUTH_NUMBER', index=['is_autoapp', 'is_den'], aggfunc='count', margins=True)

HCP_CONNECT_AUTH_NUMBER
is_autoapp is_den                         
0          0                        631695
           1                         16858
1          0                        277239
All                                 925792

In [232]:
refs_new.drop(labels=['index', 'REF_TYPE_KEY', 'PPL', 'Date_Received', 'Date_Decision', 'status_cat'], axis=1, inplace=True)

In [233]:
14849/903241

0.016439687746681118

## Create Refs_manual

In [234]:
## This is a cpt_code level list of all manually reviewed referrals

In [235]:
refs_manual = refs[refs['is_autoapp']==0]

In [236]:
refs_manual.pivot_table(values='HCP_CONNECT_AUTH_NUMBER', index=['is_autoapp', 'is_den'], aggfunc='count', margins=True)

HCP_CONNECT_AUTH_NUMBER
is_autoapp is_den                         
0          0                       2485581
           1                         59086
All                                2544667

In [237]:
## Calculate Cost per manually reviewed CPT code

ga_cpt = 6500000 / refs[refs['is_autoapp']==0].shape[0]

In [238]:
ga_cpt

2.530743668760695

## Create Refssum_manual

In [239]:
## This is a grouped list of refs_manual that summarizes the volume of manually reviewed cpt_codes

In [240]:
refssum_manual = refs_manual.groupby(['Specialty', 'CPT_Code', 'is_PPL'], as_index=False).agg({
    'UNITS' : 'count'
})

In [241]:
refssum_manual.rename(index=str, columns={'UNITS': 'UNITS_man'}, inplace=True)

In [242]:
## refssum_manual.rename(index=str, columns={'NumberOfUserEventsNoViews': 'mean_touches_manual'}, inplace=True)

In [243]:
refssum_manual['cost_to_review'] = refssum_manual['UNITS_man']*ga_cpt

In [244]:
refssum_manual.head()

,Specialty,CPT_Code,is_PPL,UNITS_man,cost_to_review
0,ACUPUNCTURE,20550,1,1,2.530744
1,ACUPUNCTURE,20552,1,2,5.061487
2,ACUPUNCTURE,20610,1,2,5.061487
3,ACUPUNCTURE,2101,0,12,30.368924
4,ACUPUNCTURE,2101,1,7,17.715206


## Create Refssum

In [245]:
## This is a grouped list of 

In [246]:
refssum = refs.groupby(['Specialty', 'CPT_Code', 'is_PPL'], as_index=False).agg({
    'UNITS': 'count',
    'is_autoapp': 'mean',
    'is_den': 'mean',
})

In [247]:
refssum.head()

,Specialty,CPT_Code,is_PPL,UNITS,is_autoapp,is_den
0,ACUPUNCTURE,20550,1,1,0.0,0.000000
1,ACUPUNCTURE,20552,1,2,0.0,0.000000
2,ACUPUNCTURE,20610,1,2,0.0,0.000000
3,ACUPUNCTURE,2101,0,12,0.0,1.000000
4,ACUPUNCTURE,2101,1,7,0.0,0.857143


In [248]:
refs_w_claims0 = pd.merge(refssum, refssum_manual, on=['Specialty', 'CPT_Code', 'is_PPL'], how='left')

In [249]:
refs_w_claims0.head()

,Specialty,CPT_Code,is_PPL,UNITS,is_autoapp,is_den,UNITS_man,cost_to_review
0,ACUPUNCTURE,20550,1,1,0.0,0.000000,1.0,2.530744
1,ACUPUNCTURE,20552,1,2,0.0,0.000000,2.0,5.061487
2,ACUPUNCTURE,20610,1,2,0.0,0.000000,2.0,5.061487
3,ACUPUNCTURE,2101,0,12,0.0,1.000000,12.0,30.368924
4,ACUPUNCTURE,2101,1,7,0.0,0.857143,7.0,17.715206


In [250]:
claims.head()

,Specialty,CPT_Code,cnt_hcp_cost,avg_hcp_cost,sd_hcp_cost
0,ACUPUNCTURE,97010,9,65.0000,0.000000
1,ACUPUNCTURE,97016,171,19.5935,16.730112
2,ACUPUNCTURE,97024,1,10.0000,NaN
3,ACUPUNCTURE,97026,252,4.6179,1.371409
4,ACUPUNCTURE,97110,7,41.4285,7.480132


In [251]:
refs_w_claims1 = pd.merge(refs_w_claims0, claims, on=['Specialty', 'CPT_Code'], how='left')

In [252]:
refs_w_claims1[refs_w_claims1['Specialty']=='CARDIOLOGY'].head()

,Specialty,CPT_Code,is_PPL,UNITS,is_autoapp,is_den,UNITS_man,cost_to_review,cnt_hcp_cost,avg_hcp_cost,sd_hcp_cost
3748,CARDIOLOGY,0095T,0,1,0.000000,0.000000,1.0,2.530744,NaN,NaN,NaN
3749,CARDIOLOGY,01922,0,2,0.000000,0.000000,2.0,5.061487,NaN,NaN,NaN
3750,CARDIOLOGY,0295T,0,51,0.000000,0.000000,51.0,129.067927,4.0,73.4375,18.880290
3751,CARDIOLOGY,0295T,1,132,0.833333,0.007576,22.0,55.676361,4.0,73.4375,18.880290
3752,CARDIOLOGY,0296T,0,100,0.000000,0.000000,100.0,253.074367,38.0,63.7007,204.695494


In [253]:
## For spec/cpt combos that don't have claims data associated, use the average of that cpt across specialties
refs_w_claims2 = pd.merge(refs_w_claims1, claims_sum, on='CPT_Code', how='left')

In [254]:
refs_w_claims2['avg_hcp_cost_x'] = np.where(refs_w_claims2['avg_hcp_cost_x'].isnull(), 
                                             refs_w_claims2['avg_hcp_cost_y'],
                                             refs_w_claims2['avg_hcp_cost_x'])

In [255]:
refs_w_claims2.drop(columns=['avg_hcp_cost_y', 'sd_hcp_cost'], inplace=True)

In [256]:
refs_w_claims2.rename(index=str, columns={'avg_hcp_cost_x': 'avg_hcp_cost'}, inplace=True)

In [257]:
refs_w_claims2[refs_w_claims2['Specialty']=='CARDIOLOGY'].head()

,Specialty,CPT_Code,is_PPL,UNITS,is_autoapp,is_den,UNITS_man,cost_to_review,cnt_hcp_cost,avg_hcp_cost
3748,CARDIOLOGY,0095T,0,1,0.000000,0.000000,1.0,2.530744,NaN,NaN
3749,CARDIOLOGY,01922,0,2,0.000000,0.000000,2.0,5.061487,NaN,604.416083
3750,CARDIOLOGY,0295T,0,51,0.000000,0.000000,51.0,129.067927,4.0,73.437500
3751,CARDIOLOGY,0295T,1,132,0.833333,0.007576,22.0,55.676361,4.0,73.437500
3752,CARDIOLOGY,0296T,0,100,0.000000,0.000000,100.0,253.074367,38.0,63.700700


In [258]:
refs_w_claims_fin = refs_w_claims2

In [259]:
refs_w_claims_fin[(refs_w_claims_fin['UNITS_man'].isnull())].shape[0]

1687

## Calculate ROI

In [260]:
refs_w_claims_fin['sum_cost_denied'] = refs_w_claims_fin['is_den']*refs_w_claims_fin['UNITS']*refs_w_claims_fin['avg_hcp_cost']

In [261]:
refs_w_claims_fin['UNITS_man'] = np.where(refs_w_claims_fin['UNITS_man'].isnull(), 0, refs_w_claims_fin['UNITS_man'])

In [262]:
refs_w_claims_fin['ROI'] = refs_w_claims_fin['sum_cost_denied']/refs_w_claims_fin['cost_to_review']

In [263]:
refs_w_claims_fin[(refs_w_claims_fin['ROI'].isnull()) &
                 (refs_w_claims_fin['Specialty']=='CARDIOLOGY')].head(15)

,Specialty,CPT_Code,is_PPL,UNITS,is_autoapp,is_den,UNITS_man,cost_to_review,cnt_hcp_cost,avg_hcp_cost,sum_cost_denied,ROI
3748,CARDIOLOGY,0095T,0,1,0.0,0.0,1.0,2.530744,NaN,NaN,NaN,NaN
3760,CARDIOLOGY,0389T,1,1,1.0,0.0,0.0,NaN,2.0,51.31500,0.0,NaN
3763,CARDIOLOGY,0399T,1,4,1.0,0.0,0.0,NaN,NaN,456.35500,0.0,NaN
3764,CARDIOLOGY,0545F,1,1,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
3765,CARDIOLOGY,1111F,1,1,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
3768,CARDIOLOGY,300,0,0,0.0,0.0,0.0,0.000000,NaN,308.81595,0.0,NaN
3769,CARDIOLOGY,3120F,1,2,0.0,0.0,2.0,5.061487,NaN,NaN,NaN,NaN
3780,CARDIOLOGY,33220,0,1,0.0,0.0,1.0,2.530744,NaN,NaN,NaN,NaN
3804,CARDIOLOGY,33271,0,1,0.0,0.0,1.0,2.530744,NaN,NaN,NaN,NaN
3806,CARDIOLOGY,33273,0,1,0.0,0.0,1.0,2.530744,NaN,NaN,NaN,NaN


In [264]:
refs_w_claims_fin[(refs_w_claims_fin['ROI'].isnull())]['UNITS'].sum()

20674

In [265]:
## For groups where we don't know the average cost from 2018, the denominator of ROI is 0, and ROI is undefined. 
## Update the ROI for those to = 100 so they are NOT included in the dictionaries to auto-approve going forward.
refs_w_claims_fin['ROI'] = np.where(refs_w_claims_fin['avg_hcp_cost'].isnull(), 100, refs_w_claims_fin['ROI'])

In [266]:
## For groups that were auto-approved at 100%, the denominator of ROI is 0, and ROI is undefined. 
## Update the ROI for those to = 0 so they are included in the dictionaries to auto-approve going forward.
refs_w_claims_fin['ROI'] = np.where(refs_w_claims_fin['ROI'].isnull(), 0.01, refs_w_claims_fin['ROI'])
        

In [267]:
refs_w_claims_fin['cost_to_review'].sum()

6205806.109993908

In [268]:
refs_w_claims_fin['fin_aa_rec'] = np.where(refs_w_claims_fin['ROI']<1, 1, 0)

## Create Dictionaries

In [269]:
## Model Additional Auto Approvals when different thresholds are set.
## Approach: use "given" threshold to determine which CPT codes are "auto-approve"-able for each specialty
##  - For loop through referrals, return 1 if all CPT codes are on "auto-approve"-able list, else 0 

In [270]:
list_o_specs = list(set(refs['Specialty'].unique().tolist() + refs_new['Specialty'].unique().tolist()))

In [271]:
#v1_codes_to_approve = pd.read_csv('../Data/V1_codes_to_approve.csv')

In [272]:
list_o_types_to_pend = ['INPT ADM', 'EMERGENCY ROOM', 'DAY SURG', 'OOA INPT', 
                        'SKILLED NURSING', 'OBSERVATION', 'OB OBSERVATION']

In [273]:
## Function that gathers CPT codes that are auto-approvable and stores them in a dictionary.
def create_dict_of_CPT_codes(specialty_cpt, list_o_specs, PPL=1):
    spec_dict = {k: [] for k in list_o_specs}
    for index, row in specialty_cpt.iterrows():
        if row['fin_aa_rec'] == 1:
            if row['is_PPL'] == PPL:
                if row['UNITS'] > 30:
                    spec_dict[row['Specialty']].append(row['CPT_Code'])
    return spec_dict
            

In [274]:
## Function that gathers CPT codes that should be pended and stores them in a dictionary.
def create_dict_of_CPT_codes_v2(specialty_cpt, list_o_specs, PPL=1):
    spec_dict = {k: [] for k in list_o_specs}
    for index, row in specialty_cpt.iterrows():
        if row['fin_aa_rec'] == 0:
            if row['is_PPL'] == PPL:
                spec_dict[row['Specialty']].append(row['CPT_Code'])
    return spec_dict
            

In [275]:
#spec_dict_PPL = create_dict_of_CPT_codes(v1_codes_to_approve, list_o_specs)

In [276]:
spec_dict_PPL = create_dict_of_CPT_codes(refs_w_claims_fin, list_o_specs)

In [277]:
spec_dict_EPL = {k: [] for k in list_o_specs}

In [278]:
spec_dict_EPL = create_dict_of_CPT_codes(refs_w_claims_fin, list_o_specs, PPL=0)

In [279]:
spec_dict_PPL['ENDOCRINOLOGY']

['10022',
 '36416',
 '76536',
 '76942',
 '81002',
 '82947',
 '82962',
 '83036',
 '95250',
 '95251',
 '96372',
 '99091',
 '99203',
 '99204',
 '99213',
 '99214',
 '99215',
 'J3489',
 'Z7500']

In [280]:
## Create dictionaries to store added and removed codes for EPL and PPL dictionaries
spec_dict_EPL_added = spec_dict = {k: [] for k in list_o_specs}
spec_dict_EPL_removed = spec_dict = {k: [] for k in list_o_specs}
spec_dict_PPL_added = spec_dict = {k: [] for k in list_o_specs}
spec_dict_PPL_removed = spec_dict = {k: [] for k in list_o_specs}

In [281]:
## Remove from EPL lists codes that don't appear in PPL list. 
## Spirit of this is to keep EPL lists more conservative
for spec in list_o_specs:
    epl_list = spec_dict_EPL[spec]
    ppl_list = spec_dict_PPL[spec]
    new_epl = [x for x in epl_list if x in ppl_list]
    spec_dict_EPL[spec] = new_epl

In [282]:
refs_w_claims_fin[(refs_w_claims_fin['Specialty']=='PAIN MANAGEMENT') &
                 (refs_w_claims_fin['is_PPL']==1) &
                 (refs_w_claims_fin['fin_aa_rec']==1) &
                 (refs_w_claims_fin['UNITS']>30)].head()

,Specialty,CPT_Code,is_PPL,UNITS,is_autoapp,is_den,UNITS_man,cost_to_review,cnt_hcp_cost,avg_hcp_cost,sum_cost_denied,ROI,fin_aa_rec
29706,PAIN MANAGEMENT,20550,1,37,0.0,0.000000,37.0,93.637516,37.0,51.6329,0.000000,0.000000,1
29710,PAIN MANAGEMENT,20552,1,188,0.0,0.000000,188.0,475.779810,126.0,55.0380,0.000000,0.000000,1
29712,PAIN MANAGEMENT,20553,1,296,0.0,0.000000,296.0,749.100126,909.0,80.7839,0.000000,0.000000,1
29720,PAIN MANAGEMENT,20610,1,344,0.0,0.002899,344.0,870.575822,718.0,61.5708,61.392334,0.070519,1
29722,PAIN MANAGEMENT,20611,1,31,0.0,0.000000,31.0,78.453054,98.0,76.9865,0.000000,0.000000,1


## Define codes to ADD or REMOVE from each list

In [283]:
## ADD Derm codes - provided my Christel McRae (124)
derm_add0 = ['10060','10061','11000','11055','11057','11100','11101','11300','11301','11302','11303','11305',
            '11306','11307','11308','11310','11311','11312','11313','11404',
            '11426','11440','11600',
            '11601','11602','11603','11604','11606','11620','11621','11622','11623','11624','11626','11640',
            '11641','11642','11643','11644','11646','11900','11901','29580','54050','54056','54100','67810',
            '69100','87101','87220','J3301','10061','10080','10081','10140','10180','11055','11056','11057',
            '11101','11300','11301','11302','11303','11305','11306','11307','11308','11310','11311','11312',
            '11313','11404',
            '11426','11440','11444','11446','11600','11601','11602','11603','11604',
            '11606','11620','11621','11622','11623','11624','11626','11640','11641','11642','11643','11644',
            '11646','11900','11901','54105']


In [284]:
1379 + 124 

1503

In [285]:
## find unique values (72)
derm_add = list(set(derm_add0))

In [286]:
## ADD Pain Add for PPL
pain_add0 = ['99203','99204','99213','99214']

In [287]:
## ADD Gastro and General Surgery for PPL and EPL
gi_add0 = ['45378','45380','45385','G0105','G0121','00812']

In [288]:
## ADD Blood Transfusions - across all specialties PPL and EPL
bt_add0 = ['36430','86900','86901','86902','86903','86904','86905','86906','86907','86908','86909','86910'
          ,'86911','86912','86913','86914','86915','86916','86917','86918','86919','86920']

In [289]:
## ADD Podiatry
podiatry_add0 = ['11055']

In [290]:
## REMOVE from all dictionaries
codes_to_remove0 = ['97810','95115','95116','95117','95170','95180','95181','95182','95183','95184','95185','95186',
                   '95187','95188','95199','95004','A0999','A0426','A0427','A0428','A0429','92590','92591','92592',
                   '92593','92594','90901','90902','90903','90904','90905','90906','90907','90908','90909','90910',
                   '90911','90875','90876','86890','93798','98940','J1050','58301','58565','58611','G0337',
                   '74740','89320','J3490','J0717','J1438','J7321','J7322','J7323','J7324','J7325','J3489','J0135',
                   'J3030','J1830','J9215','J9218','J0129','J9202','J3301','J1745','96365','96366','90378','J2505',
                   'J1440','J1441','J0885','J0886','T1013','96118','96119','96120','97166','97167','97168','97110',
                   '92015','97162','97163','97110','36468','92506','55250','59840',
                   'S0199','S0618','92595','58300','J7300','57170','A4266','11981',
                   'J7302','J7300','58670','59600','58605','64612','J0585','11055',
                   '11057','11719','11721','G0127','S0390']

In [291]:
codes_to_remove = list(set(codes_to_remove0))

In [292]:
codes_to_pend.head()

,Specialty,CPT_Code,fin_aa_rec,is_PPL
0,CARDIOLOGY,93228,0,1
1,CARDIOLOGY,93229,0,1
2,CARDIOLOGY,0297T,0,1
3,CARDIOLOGY,0298T,0,1
4,ENT-OTOLARYNGOLOGY,31237,0,1


In [293]:
codes_to_pend[codes_to_pend['Specialty']=='ENDOCRINOLOGY']

,Specialty,CPT_Code,fin_aa_rec,is_PPL
474,ENDOCRINOLOGY,36416,0,1
475,ENDOCRINOLOGY,81002,0,1
476,ENDOCRINOLOGY,82947,0,1
477,ENDOCRINOLOGY,82962,0,1
478,ENDOCRINOLOGY,83036,0,1
479,ENDOCRINOLOGY,95250,0,1
480,ENDOCRINOLOGY,95251,0,1
481,ENDOCRINOLOGY,99091,0,1
482,ENDOCRINOLOGY,J3489,0,1
541,ENDOCRINOLOGY,99204,1,1


In [294]:
laurie_christel_dict_to_remove = create_dict_of_CPT_codes_v2(codes_to_pend, list_o_specs)

In [295]:
laurie_christel_dict_to_remove['ENDOCRINOLOGY']

['36416',
 '81002',
 '82947',
 '82962',
 '83036',
 '95250',
 '95251',
 '99091',
 'J3489']

In [296]:
list_o_specs_pend = codes_to_pend['Specialty'].unique().tolist()

In [297]:
## Define function that takes specialty:cpt code list DICTIONARY, list of cpt
## codes, and "add" or "remove" parameter and adjusts the dictionary
def add_remove_cpt_codes_v2(spec_cpt_dict, spec_cpt_dict_added_removed, spec_list, cpt_to_add_remove_dict, add=True):
    if add == True:
        for spec in spec_list:
            for code in cpt_to_add_remove_dict[spec]:
                if code in spec_cpt_dict[spec]:
                    continue
                else:
                    spec_cpt_dict[spec].append(code)
                    spec_cpt_dict_added_removed[spec].append(code)
    else:
        for spec in spec_list:
            for code in cpt_to_add_remove_dict[spec]:
                if code in spec_cpt_dict[spec]:
                    spec_cpt_dict[spec].remove(code)
                    spec_cpt_dict_added_removed[spec].append(code)
                else:
                    continue
    return spec_cpt_dict, spec_cpt_dict_added_removed        

In [298]:
## Define function that takes specialty:cpt code list, list of cpt
## codes, and "add" or "remove" parameter and adjusts the dictionary
def add_remove_cpt_codes_v1(spec_cpt_dict, spec_cpt_dict_added_removed, spec_list, cpt_to_add_remove, add=True):
    if add == True:
        for spec in spec_list:
            for code in cpt_to_add_remove:
                if code in spec_cpt_dict[spec]:
                    continue
                else:
                    spec_cpt_dict[spec].append(code)
                    spec_cpt_dict_added_removed[spec].append(code)
    else:
        for spec in spec_list:
            for code in cpt_to_add_remove:
                if code in spec_cpt_dict[spec]:
                    spec_cpt_dict[spec].remove(code)
                    spec_cpt_dict_added_removed[spec].append(code)
                else:
                    continue
    return spec_cpt_dict, spec_cpt_dict_added_removed        

In [299]:
## add derm
spec_dict_PPL, spec_dict_PPL_added = add_remove_cpt_codes_v1(spec_dict_PPL, spec_dict_PPL_added, ['DERMATOLOGY'], derm_add)
spec_dict_EPL, spec_dict_EPL_added = add_remove_cpt_codes_v1(spec_dict_EPL, spec_dict_EPL_added, ['DERMATOLOGY'], derm_add)

In [300]:
## add pain
spec_dict_PPL, spec_dict_PPL_added = add_remove_cpt_codes_v1(spec_dict_PPL, spec_dict_PPL_added, ['PAIN MANAGEMENT'], pain_add0)
spec_dict_EPL, spec_dict_EPL_added = add_remove_cpt_codes_v1(spec_dict_EPL, spec_dict_EPL_added, ['PAIN MANAGEMENT'], pain_add0)

In [301]:
## add pain
spec_dict_PPL, spec_dict_PPL_added = add_remove_cpt_codes_v1(spec_dict_PPL, spec_dict_PPL_added, 
                                                          ['GASTROENTEROLOGY','SURGERY - GENERAL'], gi_add0)
spec_dict_EPL, spec_dict_EPL_added = add_remove_cpt_codes_v1(spec_dict_EPL, spec_dict_EPL_added, 
                                                          ['GASTROENTEROLOGY','SURGERY - GENERAL'], gi_add0)

In [302]:
## add pain
spec_dict_PPL, spec_dict_PPL_added = add_remove_cpt_codes_v1(spec_dict_PPL, spec_dict_PPL_added, ['PODIATRY'], podiatry_add0)
spec_dict_EPL, spec_dict_EPL_added = add_remove_cpt_codes_v1(spec_dict_EPL, spec_dict_EPL_added, ['PODIATRY'], podiatry_add0)

In [303]:
## remove codes from PPL
spec_dict_PPL, spec_dict_PPL_removed = add_remove_cpt_codes_v1(spec_dict_PPL, spec_dict_PPL_removed, list_o_specs, codes_to_remove, add=False)

In [304]:
## remove codes from EPL
spec_dict_EPL, spec_dict_EPL_removed = add_remove_cpt_codes_v1(spec_dict_EPL, spec_dict_EPL_removed, list_o_specs, codes_to_remove, add=False)

In [305]:
## Remove codes from dictionaries as chosen by Laurie and Christel
spec_dict_PPL, spec_dict_PPL_removed = add_remove_cpt_codes_v2(spec_dict_PPL, spec_dict_PPL_removed, list_o_specs_pend, laurie_christel_dict_to_remove, add=False)

In [306]:
spec_dict_PPL['ENDOCRINOLOGY']

['10022',
 '76536',
 '76942',
 '96372',
 '99203',
 '99204',
 '99213',
 '99214',
 '99215',
 'Z7500']

In [307]:
list_o_specs

['PODIATRY',
 'NUCLEAR MEDICINE',
 'PATHOLOGY',
 'PSYCHOLOGY',
 'RADIOLOGY',
 'NEPHROLOGY',
 'MFCC (THERAPIST)',
 'PEDS-DEVELOPMENTAL BEHAVIORAL',
 'CLINICAL PSYCHOLOGIST',
 'SURGERY - THORACIC',
 'DIALYSIS',
 'CARDIOLOGY',
 'PHARMACY',
 'SURGERY - VASCULAR',
 'ENT-OTOLARYNGOLOGY',
 'ORTHOTICS AND PROSTHETICS',
 'PALLIATIVE CARE',
 'HEPATOLOGY',
 'ONCOLOGY - MEDICAL',
 'GERIATRICS',
 'NEONATOLOGY',
 'EMERGENCY SERVICES',
 'SURGERY - HAND',
 'FACILITY SERVICES',
 'INFERTILITY',
 'INFECTIOUS DISEASE',
 'MED SUPP COMP WITH PHARMACIST',
 'SUPPLIERS - ALL OTHER',
 'SURGERY - PLASTIC/RECONSTRUCT',
 'SURGERY - ONCOLOGY',
 'CERTIFIED NURSE MIDWIFE',
 'HOSPITALIST',
 'SURGERY - ORAL',
 'ADDICTION MEDICINE',
 'PUBLIC HEALTH/ WELFARE AGENCY',
 'OPTICIAN',
 'OCCUPATIONAL THERAPY',
 'ONCOLOGY - GYN',
 'ACUPUNCTURE',
 'NURSING FACILITY - OTHER',
 'INDIV CERTIFIED ORTHOTIST',
 'PAIN MANAGEMENT',
 'LICENSED CLIN SOCIAL WORKER',
 'INFUSION THERAPY',
 'GASTROENTEROLOGY',
 'SURGERY - COLONRECTAL',
 'DENT

In [308]:
## For specialties that UM clinical team deem necessary, empty dictionary
## To ensure that no referrals are auto-approved from it
specs_that_should_pend = ['ACUPUNCTURE', 'ADDICTION MEDICINE', 'ANESTHESIOLOGY',
                         'BEHAVIORAL HEALTH', 'CHIROPRACTIC', 'DENTIST', 'DME MAINTENANCE',
                         'LICENSED CLIN SOCIAL WORKER', 'MFCC (THERAPIST)', 'NON-CONTRACT UNKWN BILL AREA',
                         'NURSE PRACTITIONER', 'NURSING FACILITY - OTHER', 'OPTICIAN',
                        'PEDS-DEVELOPMENTAL BEHAVIORAL', 'PSYCHIATRY', 'PSYCHOLOGY', 'SENIOR WELLNESS VISIT',
                         'SPORTS MEDICINE', 'SURGERY - ORAL', 'AMBULATORY SURGICAL CENTER', 'AMBULANCE', 'ALLERGY/IMMUNOLOGY',
                         'CUSTODIAL CARE', 'HOME HEALTH', 'INFERTILITY', 'OPTOMETRY', 'PALLIATIVE CARE', 'PODIATRY', 'SLEEP STUDY',
                         'SNF - FAC', 'OCCUPATIONAL THERAPY', 'PHYSICAL THERAPY/REHAB']

In [309]:
def pend_specs_by_spec(list_o_specs, spec_dict):
    for spec in list_o_specs:
        spec_dict[spec] = []
    return spec_dict

In [310]:
spec_dict_PPL = pend_specs_by_spec(specs_that_should_pend, spec_dict_PPL)

In [311]:
spec_dict_EPL = pend_specs_by_spec(specs_that_should_pend, spec_dict_EPL)

In [312]:
#### Prepare a newer set of referrals to run through the new code.

In [313]:
## Assigns an auto-approve (1) or pend (0) status to each specialty / code group
def assign_status(codes, spec_dict_PPL, spec_dict_EPL, list_of_types_pend):
    status = list(np.zeros(codes.shape[0]))
    specs_list = codes['Specialty'].unique().tolist()
    print(len(status))
    print(codes.shape[0])
    for spec in specs_list:
        if spec not in spec_dict_PPL:
            spec_dict_PPL[spec] = []
    for spec in specs_list:
        if spec not in spec_dict_EPL:
            spec_dict_EPL[spec] = []
    for index, row in codes.iterrows():
        if row['is_PPL'] == 1:
            if row['CPT_Code'] in spec_dict_PPL[row['Specialty']]:
                if row['Type'] not in list_of_types_pend:
                    status[index] = 1
        ##else:
          ##  if row['CPT_Code'] in spec_dict_EPL[row['Specialty']]:
            ##    status[index] = 0
    return status

In [314]:
auto_approve = assign_status(refs_new, spec_dict_PPL, 
                             spec_dict_EPL, list_o_types_to_pend)

925792
925792


In [315]:
refs_new['auto_approvable'] = auto_approve

In [316]:
refs_new.head()

,CPT_Code,HCP_CONNECT_AUTH_NUMBER,Type,status_name,Specialty,region,UNITS,is_autoapp,is_PPL,is_app,is_den,auto_approvable
0,99213,16132990H,PHYSICIAN,APPROVED - AUTO,OBSTETRICS/GYNECOLOGY,SOUTH BAY,1,1,1,1,0,1.0
1,99214,16113572H,PHYSICIAN,APPROVED - CARE COORDINATOR,SURGERY - NEURO,SAN GABRIEL VALLEY,1,0,1,1,0,1.0
2,45380,16121057H,OUTPT DIAG,APPROVED - CM,FACILITY SERVICES,SOUTH BAY,1,0,1,1,0,1.0
3,72040,16116056H,OUTPT DIAG,APPROVED - CARE COORDINATOR,RADIOLOGY,SAN FERNANDO VALLEY,1,0,1,1,0,1.0
4,31231,16120070H,PHYSICIAN,APPROVED - MD,FACILITY SERVICES,ORANGE COUNTY,1,0,0,1,0,0.0


In [317]:
## Find which (REFERRALS) not CPT codes have all cpt codes in the dictionary and thus, "auto-approvable"

In [318]:
refs_results = refs_new.groupby(['HCP_CONNECT_AUTH_NUMBER'], as_index=False).agg({'auto_approvable': 'mean'})

In [319]:
refs_results['aa-yn'] = np.where(refs_results['auto_approvable']==1, 1, 0)

In [320]:
refs_results.drop('auto_approvable', axis=1, inplace=True)

In [321]:
refs_results.tail(10)

,HCP_CONNECT_AUTH_NUMBER,aa-yn
550984,16505584H,0
550985,16506737H,0
550986,16507905H,0
550987,16513452H,0
550988,16513736H,1
550989,16528009H,0
550990,16555364H,1
550991,16586687H,1
550992,16602597H,0
550993,82728,0


In [322]:
# refs_w_claims_dec = refs_w_claims_fin.drop(labels=['UNITS', 'is_autoapp', 'is_den', 'UNITS_man',
#                               'cost_to_review', 'cnt_hcp_cost', 'sum_cost_denied'], axis=1)

In [323]:
refs_new.head()

,CPT_Code,HCP_CONNECT_AUTH_NUMBER,Type,status_name,Specialty,region,UNITS,is_autoapp,is_PPL,is_app,is_den,auto_approvable
0,99213,16132990H,PHYSICIAN,APPROVED - AUTO,OBSTETRICS/GYNECOLOGY,SOUTH BAY,1,1,1,1,0,1.0
1,99214,16113572H,PHYSICIAN,APPROVED - CARE COORDINATOR,SURGERY - NEURO,SAN GABRIEL VALLEY,1,0,1,1,0,1.0
2,45380,16121057H,OUTPT DIAG,APPROVED - CM,FACILITY SERVICES,SOUTH BAY,1,0,1,1,0,1.0
3,72040,16116056H,OUTPT DIAG,APPROVED - CARE COORDINATOR,RADIOLOGY,SAN FERNANDO VALLEY,1,0,1,1,0,1.0
4,31231,16120070H,PHYSICIAN,APPROVED - MD,FACILITY SERVICES,ORANGE COUNTY,1,0,0,1,0,0.0


In [324]:
refs_new.shape[0]

925792

In [325]:
refs_new = refs_new.merge(refs_results, how='left', on=['HCP_CONNECT_AUTH_NUMBER'])

In [326]:
refs_w_claims_fin[refs_w_claims_fin['CPT_Code']=='J1561'].head()

,Specialty,CPT_Code,is_PPL,UNITS,is_autoapp,is_den,UNITS_man,cost_to_review,cnt_hcp_cost,avg_hcp_cost,sum_cost_denied,ROI,fin_aa_rec
16852,FACILITY SERVICES,J1561,0,12,0.0,0.250000,12.0,30.368924,105.0,2019.270300,6057.810900,199.474005,0
16853,FACILITY SERVICES,J1561,1,1,0.0,0.000000,1.0,2.530744,105.0,2019.270300,0.000000,0.000000,1
19914,HEMATOLOGY/ONCOLOGY,J1561,0,13,0.0,0.153846,13.0,32.899668,NaN,3831.021686,7662.043371,232.891209,0
20806,HOME HEALTH,J1561,0,37,0.0,0.000000,37.0,93.637516,20.0,4668.504000,0.000000,0.000000,1
20807,HOME HEALTH,J1561,1,14,0.0,0.000000,14.0,35.430411,20.0,4668.504000,0.000000,0.000000,1


In [327]:
refs_new.pivot_table(values='UNITS', index=['is_autoapp', 'is_den'], columns='aa-yn', aggfunc='count', margins=True)##.to_csv('../Data/Outputs/costs_cm.csv')

aa-yn                   0       1     All
is_autoapp is_den                        
0          0       447012  184683  631695
           1        15799    1059   16858
1          0        81208  196031  277239
All                544019  381773  925792

In [328]:
#refs_new.to_csv('../Data/Outputs/refs_new_from_script.csv', sep='|')

In [329]:
# refs_new[(refs_new['is_autoapp']==1) &
#          (refs_new['aa-yn']==0)].pivot_table(values='UNITS', index = 'is_den',
#                     aggfunc='count', margins=True).to_csv('../Data/Outputs/2019_spec_cpt_results.csv', sep='|')

In [330]:
refs_new.columns

Index(['CPT_Code', 'HCP_CONNECT_AUTH_NUMBER', 'Type', 'status_name',
       'Specialty', 'region', 'UNITS', 'is_autoapp', 'is_PPL', 'is_app',
       'is_den', 'auto_approvable', 'aa-yn'],
      dtype='object')

In [331]:
# refs_new.pivot_table(values ='CPT_Code', index=['Type','auto_approvable'], columns='is_app'
#                     ,aggfunc='count', margins=True).to_csv('../Data/Outputs/2019_project_cm_type.csv', sep='|')

In [332]:
refs_new.shape[0]

925792

In [333]:
len(refs_new['HCP_CONNECT_AUTH_NUMBER'].unique())

550994

In [334]:
refs_new.columns

Index(['CPT_Code', 'HCP_CONNECT_AUTH_NUMBER', 'Type', 'status_name',
       'Specialty', 'region', 'UNITS', 'is_autoapp', 'is_PPL', 'is_app',
       'is_den', 'auto_approvable', 'aa-yn'],
      dtype='object')

## Construct Specialty Summary

In [335]:
refs_new_head = refs_new.groupby(['Specialty', 'Type', 'region', 'HCP_CONNECT_AUTH_NUMBER'], as_index=False).agg({
    'UNITS': 'sum',
    'is_autoapp': 'mean',
    'is_PPL': 'mean',
    'is_app': 'mean',
    'is_den': 'mean',
    'aa-yn': 'mean'
})

In [336]:
refs_new_head.pivot_table(values=['is_autoapp', 'aa-yn'], index='Specialty', aggfunc=['count', 'mean']).to_csv('../Data/Outputs/spec_rates.csv',
                                                                                                              sep='|')

In [337]:
specialty_summary = pd.read_csv('../Data/Outputs/spec_rates.csv', sep='|', skiprows=3, names=['Specialty',
                                                                                             'ref_vol', 'ref_vol2',
                                                                                             'new_rate', 'old_rate'])

In [338]:
specialty_summary.drop('ref_vol2', axis=1, inplace=True)

In [339]:
## Get counts of codes for each specialty

In [340]:
specialty_summary.tail()

,Specialty,ref_vol,new_rate,old_rate
101,SURGERY - VASCULAR,4050,0.295802,0.000247
102,UNK PHYS SPEC,1485,0.000000,0.005387
103,URGENT/IMMEDIATE CARE,99,0.121212,0.414141
104,UROLOGY,17400,0.695460,0.584368
105,WOUND CARE,1153,0.011275,0.267129


In [341]:
def spec_code_counts(spec_dict, spec_summary):
    counts = []
    for index, row in spec_summary.iterrows():
        counts.append(len(spec_dict[row['Specialty']]))
    return counts

In [342]:
PPL_counts = spec_code_counts(spec_dict_PPL, specialty_summary)

In [343]:
specialty_summary['num_codes']=PPL_counts

In [344]:
## get 'denied now approved'

In [345]:
refs_new_head['denied_now_aa'] = refs_new_head['is_den'] * refs_new_head['aa-yn']

In [346]:
refs_new_head.pivot_table('denied_now_aa', index='Specialty', aggfunc='sum').to_csv('../Data/Outputs/spec_den_now_aa.csv',
                                                                                                      sep='|')

In [347]:
ss_deny_now_aa = pd.read_csv('../Data/Outputs/spec_den_now_aa.csv', sep='|', skiprows=2, names=['Specialty', 'deny_now_aa'])

In [348]:
specialty_summary = specialty_summary.merge(ss_deny_now_aa, how='inner', on='Specialty')

In [349]:
specialty_summary.head()

,Specialty,ref_vol,new_rate,old_rate,num_codes,deny_now_aa
0,ADDICTION MEDICINE,114,0.0,0.000000,0,0
1,ALLERGY/IMMUNOLOGY,4048,0.0,0.650198,0,0
2,AMBULANCE,6264,0.0,0.064974,0,0
3,AMBULATORY SURGICAL CENTER,12102,0.0,0.000083,0,0
4,ANESTHESIOLOGY,48,0.0,0.000000,0,0


In [350]:
## Prep refs_w_claims_fin to merge into refs_new

In [351]:
refs_w_claims_fin_select = refs_w_claims_fin.drop(['UNITS', 'is_autoapp', 'is_den',
       'UNITS_man', 'cost_to_review', 'cnt_hcp_cost',
       'sum_cost_denied'], axis=1)

In [352]:
refs_new_w_fin = pd.merge(refs_new, refs_w_claims_fin_select, how='left', on=['Specialty', 'CPT_Code', 'is_PPL'])

In [353]:
refs_new_w_fin['dollars_denied_now_app'] = refs_new_w_fin['is_den']*refs_new_w_fin['aa-yn']*refs_new_w_fin['avg_hcp_cost']

In [354]:
refs_new_w_fin['cost_to_review_bene'] = (1-refs_new_w_fin['is_autoapp'])*refs_new_w_fin['aa-yn']*ga_cpt

In [355]:
refs_new_w_fin['cost_to_review_loss'] = refs_new_w_fin['is_autoapp']*(1-refs_new_w_fin['aa-yn'])*ga_cpt

In [356]:
refs_new_w_fin['new_denial_bene_est'] = refs_new_w_fin['is_autoapp']*(1-refs_new_w_fin['aa-yn'])*ga_cpt*refs_new_w_fin['ROI']*refs_new_w_fin['is_PPL']

In [357]:
refs_new_w_fin.pivot_table(values=['dollars_denied_now_app', 'cost_to_review_bene', 'cost_to_review_loss',
                                  'new_denial_bene_est'], index='Specialty', aggfunc='sum').to_csv('../Data/Outputs/specialty_fins.csv', sep='|')

In [358]:
specialty_fins = pd.read_csv('../Data/Outputs/specialty_fins.csv', sep='|')

In [359]:
specialty_summary = specialty_summary.merge(specialty_fins, how='inner', on='Specialty')

In [360]:
specialty_summary['net_benefit'] = specialty_summary['cost_to_review_bene'] - specialty_summary['cost_to_review_loss'] - specialty_summary['dollars_denied_now_app']

In [361]:
specialty_summary['net_benefit_w_new_den'] = specialty_summary['cost_to_review_bene'] - specialty_summary['cost_to_review_loss'] -specialty_summary['dollars_denied_now_app'] + specialty_summary['new_denial_bene_est']

In [362]:
# Give the filename you wish to save the file to
spec_summary_filename = '../Data/Outputs/specialty_summary_fin.csv'

# Use this function to search for any files which match your filename
files_present = os.path.isfile(spec_summary_filename)

# if no matching files, write to csv, if there are matching files, print statement
if not files_present:
    specialty_summary.to_csv(refs_w_claims_fin_filename)
else:
    print('WARNING: This file already exists!')

In [363]:
refs_results.columns

Index(['HCP_CONNECT_AUTH_NUMBER', 'aa-yn'], dtype='object')

In [364]:
refs_results['aa-yn'].mean()

0.45953131976028777

In [365]:
## rate before laurie and chrisel's codes to pend
v0 = 1198038/2086468
print(v0)

0.5741942843120527


In [366]:
## rate after laurie and christels' codes to pend
v1 = 1091109/2086468
print(v1)

0.5229454753200145


In [367]:
v2 = 1097619/2086468
print(v2)

0.5260655806846786


In [368]:
## using 2018 data to create rules, and then applying them to early 2019 data
v3 = 274250/543258
print(v3)

0.5048245953119881


In [369]:
## using clinical review list from 20190501 from Rhoda 
.401

0.401

In [370]:
## 20190503 V1 - More Conservative Version
.4532

0.4532

In [371]:
## 20190503 v2 - Baseline
.462

0.462

## Code Detail List

In [372]:
refs_w_claims_fin[(refs_w_claims_fin['Specialty']=='PAIN MANAGEMENT')
                 ##& (refs_w_claims_fin['UNITS']>30)
                  & (refs_w_claims_fin['CPT_Code']=='J1885')
                 ].to_csv('../Data/Outputs/code_detail_20190411.csv', sep='|')

## Spot Check Specific Referrals

In [373]:
refs_new_head[(refs_new_head['Specialty']=='OCCUPATIONAL THERAPY') 
             ##& (refs_results['status_name']=='DENIED - CM')
            & (refs_new_head['aa-yn']==1)].tail(10)

,Specialty,Type,region,HCP_CONNECT_AUTH_NUMBER,UNITS,is_autoapp,is_PPL,is_app,is_den,aa-yn,denied_now_aa


## Referral Detail Lookup (Enter Auth)

In [374]:
refs_new[refs_new['HCP_CONNECT_AUTH_NUMBER']=='14040476H']

,CPT_Code,HCP_CONNECT_AUTH_NUMBER,Type,status_name,Specialty,region,UNITS,is_autoapp,is_PPL,is_app,is_den,auto_approvable,aa-yn


## Auth Lookup (Enter Specialty and Code)

In [375]:
refs_new[(refs_new['Specialty']=='DERMATOLOGY')
        & (refs_new['CPT_Code']=='13100')
        ##& (refs_det['auto_approvable'] == 0)
        ]

,CPT_Code,HCP_CONNECT_AUTH_NUMBER,Type,status_name,Specialty,region,UNITS,is_autoapp,is_PPL,is_app,is_den,auto_approvable,aa-yn
3019,13100,16281652H,PHYSICIAN,APPROVED - AUTO,DERMATOLOGY,SAN FERNANDO VALLEY,1,1,1,1,0,1.0,0
27983,13100,16134444H,PHYSICIAN,APPROVED - CM,DERMATOLOGY,SAN GABRIEL VALLEY,1,0,1,1,0,1.0,0
84714,13100,16133948H,PHYSICIAN,APPROVED - CM,DERMATOLOGY,SAN GABRIEL VALLEY,1,0,1,1,0,1.0,0
85487,13100,16133586H,PHYSICIAN,APPROVED - MD,DERMATOLOGY,SAN GABRIEL VALLEY,1,0,1,1,0,1.0,0
86660,13100,16133379H,PHYSICIAN,APPROVED - CM,DERMATOLOGY,SAN GABRIEL VALLEY,1,0,1,1,0,1.0,0
104595,13100,16034834H,PHYSICIAN,APPROVED - AUTO,DERMATOLOGY,SAN FERNANDO VALLEY,1,1,1,1,0,1.0,0
119816,13100,16042583H,PHYSICIAN,APPROVED - AUTO,DERMATOLOGY,LONG BEACH,1,1,1,1,0,1.0,0
132255,13100,15939071H,PHYSICIAN,APPROVED - CM,DERMATOLOGY,SAN GABRIEL VALLEY,1,0,1,1,0,1.0,0
155022,13100,15994230H,PHYSICIAN,APPROVED - CM,DERMATOLOGY,SAN GABRIEL VALLEY,1,0,1,1,0,1.0,0
186772,13100,15934963H,PHYSICIAN,APPROVED - CM,DERMATOLOGY,SAN GABRIEL VALLEY,1,0,1,1,0,1.0,0


In [376]:
def create_final_statuses(ref_summary, spec_dict_PPL):
    final_status=[]
    for index, row in ref_summary.iterrows():
        if row['is_PPL'] == 1:
            if row['CPT_Code'] in spec_dict_PPL[row['Specialty']]:
                final_status.append(1)
            else:
                final_status.append(0)
        else:
            final_status.append(0)
    return final_status

In [377]:
final_status = create_final_statuses(refs_w_claims_fin, spec_dict_PPL)

In [378]:
refs_w_claims_fin['final_status'] = final_status

In [379]:
refs_w_claims_fin.head()

,Specialty,CPT_Code,is_PPL,UNITS,is_autoapp,is_den,UNITS_man,cost_to_review,cnt_hcp_cost,avg_hcp_cost,sum_cost_denied,ROI,fin_aa_rec,final_status
0,ACUPUNCTURE,20550,1,1,0.0,0.000000,1.0,2.530744,NaN,53.542742,0.0,0.0,1,0
1,ACUPUNCTURE,20552,1,2,0.0,0.000000,2.0,5.061487,NaN,63.492905,0.0,0.0,1,0
2,ACUPUNCTURE,20610,1,2,0.0,0.000000,2.0,5.061487,NaN,144.180984,0.0,0.0,1,0
3,ACUPUNCTURE,2101,0,12,0.0,1.000000,12.0,30.368924,NaN,NaN,NaN,100.0,0,0
4,ACUPUNCTURE,2101,1,7,0.0,0.857143,7.0,17.715206,NaN,NaN,NaN,100.0,0,0


In [380]:
refs_w_claims_fin[(refs_w_claims_fin['final_status']==0) &
                 (refs_w_claims_fin['fin_aa_rec']==1)].head()

,Specialty,CPT_Code,is_PPL,UNITS,is_autoapp,is_den,UNITS_man,cost_to_review,cnt_hcp_cost,avg_hcp_cost,sum_cost_denied,ROI,fin_aa_rec,final_status
0,ACUPUNCTURE,20550,1,1,0.0,0.0,1.0,2.530744,NaN,53.542742,0.0,0.0,1,0
1,ACUPUNCTURE,20552,1,2,0.0,0.0,2.0,5.061487,NaN,63.492905,0.0,0.0,1,0
2,ACUPUNCTURE,20610,1,2,0.0,0.0,2.0,5.061487,NaN,144.180984,0.0,0.0,1,0
5,ACUPUNCTURE,27096,1,2,0.0,0.0,2.0,5.061487,NaN,283.243980,0.0,0.0,1,0
6,ACUPUNCTURE,62321,1,1,0.0,0.0,1.0,2.530744,NaN,165.405880,0.0,0.0,1,0


In [381]:
# Give the filename you wish to save the file to
refs_w_claims_fin_filename = '../Data/Outputs/spec_cpt_summary.csv'

# Use this function to search for any files which match your filename
files_present = os.path.isfile(refs_w_claims_fin_filename)

# if no matching files, write to csv, if there are matching files, print statement
if not files_present:
    refs_w_claims_fin.to_csv(refs_w_claims_fin_filename)
else:
    print('WARNING: This file already exists!')